In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from layers.RegexTokenizer import RegexTokenizer
import torch
from torch import nn
from torch.nn import functional as F
from torch.amp import autocast, GradScaler
import bitsandbytes as bnb

import time
from transformers import AutoTokenizer


from livelossplot import PlotLosses

from utils import ModelSpecs, TrainingData
from layers.Block import Block
torch.manual_seed(1337)

In [ ]:
torch.__version__

In [ ]:
# hyperparameters
modelSpecs = ModelSpecs.create('target')
# BATCH_SIZE = 38 # how many independent sequences will we process in parallel?
# BATCH_SIZE = 28 # how many independent sequences will we process in parallel?
BATCH_SIZE = 8 # how many independent sequences will we process in parallel?
MAX_ITERS = 2000
LEARNING_RATE = 3e-4
EVAL_INTERVALS = 100
# EVAL_ITERS = 200
EVAL_ITERS = 50
TRAIN_TEST_SPLIT = 0.9
# device = 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Running on {device.capitalize()} device.")


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")
stories = TrainingData.TinyStories() # TODO impliment data caching in TrainingDataing Class so that we do not read the txt file again if already read once

In [ ]:
i = 8
text = stories[int(10**6 * 0.730612 * i):int(10**6 * 0.730612 *(i+1) )]
# text = TrainingData.TinyStories()[:int(10**6 * 0.730612 * 20 * 5)]
print(len(text)/10**6,"M")


In [ ]:

data = torch.tensor(tokenizer.encode(text))

n = int(TRAIN_TEST_SPLIT * len(data))
print(len(data)/10**6,"M")

train_data = data[:n]
val_data = data[n:]


In [ ]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - modelSpecs.BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack([data[i:i+modelSpecs.BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i+1:i+modelSpecs.BLOCK_SIZE+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(modelSpecs.VOCAB_SIZE, modelSpecs.N_EMBD)
        self.position_embedding_table = nn.Embedding(modelSpecs.BLOCK_SIZE, modelSpecs.N_EMBD)
        self.blocks = nn.Sequential(*[Block(modelSpecs) for _ in range(modelSpecs.N_LAYER)])
        self.ln_f = nn.LayerNorm(modelSpecs.N_EMBD) # final layer norm
        self.lm_head = nn.Linear(modelSpecs.N_EMBD, modelSpecs.VOCAB_SIZE)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -modelSpecs.BLOCK_SIZE:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


In [ ]:
@torch.no_grad()
def estimate_loss(model : GPTLanguageModel):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(EVAL_ITERS)
        for k in range(EVAL_ITERS):
            X, Y = get_batch(split)
            with autocast(device_type='cuda', dtype=torch.float16):
                _, loss = model(X, Y)
            # _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [ ]:
m = GPTLanguageModel()
m = m.to(device)
# m = torch.compile(m, mode='default') # TODO use this when ready to use linux, and ready to train a 100M model
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
# optimizer = torch.optim.AdamW(m.parameters(), lr=LEARNING_RATE)
optimizer = bnb.optim.AdamW8bit(m.parameters(), lr=LEARNING_RATE)
scaler = GradScaler()

In [ ]:
startTime = time.time()

# temporary line
MAX_ITERS = 100
torch.cuda.empty_cache()

for iter in range(MAX_ITERS):
    print(f"iter #{iter}")
    # torch.cuda.empty_cache()
    # every once in a while evaluate the loss on train and val sets
    if iter % EVAL_INTERVALS == 0 or iter == MAX_ITERS - 1:
    #     losses = estimate_loss(model)
        # print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, time {(time.time() - startTime)/60:.2f} minutes")
        print (f"step {iter}:  time {(time.time() - startTime)/60:.2f} minutes")
    # torch.cuda.empty_cache()
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    optimizer.zero_grad(set_to_none=True)
    
    with autocast(device_type='cuda', dtype=torch.float16): 
        logits, loss = m(xb, yb)
    # logits, loss = model(xb, yb)

    scaler.scale(loss).backward()
    # loss.backward()
    scaler.step(optimizer)
    # optimizer.step()
    scaler.update()

print(torch.cuda.memory_allocated() / 1e6, "MB allocated")
print(torch.cuda.max_memory_allocated() / 1e6, "MB peak allocated")
print(torch.cuda.memory_reserved() / 1e6, "MB reserved")
# print(torch.cuda.memory_summary())


endTime = time.time()
print(f"Total Training Time : {(endTime - startTime)/60:.2f} minutes")

torch.cuda.empty_cache()

In [ ]:
# with amp : 18.18 min -- found an error here

# without amp : 3889 mib
# without amp : 3892 mib
# without amp : 3892 mib
# without amp : 3888 mib
# without amp : 9.58 min
# without amp : 8.68 min
# without amp : 8.79 min
# without amp : 8.95 min
# without amp : 2262.1056 MB allocated, 9097.445376 MB reserved

# with amp : 5.85 min 
# with amp : 6.40 min
# with amp : 6.02 min
# with amp : 3702MiB # there we might be able to icrease batch size despite it not showing in results
# with amp : 3691MiB
# with amp : 1542.184448 MB allocated, 7686.06208 MB reserved
# with amp : 1542.184448 MB allocated, 7686.06208 MB reserved

# with amp, clear_chache on only on edges (max batch 38)
# 4.01 min 1542.875648 MB allocated 7753.170944 MB reserved 3941MiB
# 4.01 min 1542.875648 6586.284032(peak) MB allocated 7753.170944 MB reserved 3941MiB

# with amp, clear_chache on only on edges 8bit-optimizer (max batch )
# 3.99 min 1245.89824 6289.961984(peak) MB allocated 7742.685184 MB reserved 3941MiB

In [ ]:
# 100M BATCH 8 BLOCK 512 16.77min 56.28days-for-2B-tokens

In [ ]:
input_tokens = torch.tensor(tokenizer.encode("there was once a forest with 100s of bushes")).unsqueeze(0).cuda()
# print(input_tokens)

m.eval()

output_tokens = m.generate(input_tokens, max_new_tokens=64)[0]
# print(output_tokens)

m.train()

output : str = tokenizer.decode(output_tokens)


print(output)

In [ ]:
print(int(len(stories)//(10**6 )))

In [ ]:
ITERS = 100
startTime = time.time()
step = BATCH_SIZE * modelSpecs.BLOCK_SIZE * ITERS * 8
liveloss = PlotLosses()

# for i in range(step * 0, len(stories), step):
for i in range(989593600, len(stories), step):
    print("ITER:", i // step, "::::", " STRING INDEX:", i)
    text = stories[i : i + step]
    data = torch.tensor(tokenizer.encode(text))
    n = int(TRAIN_TEST_SPLIT * len(data))
    print("tokens", len(data)/10**6,"M")

    train_data = data[:n]
    val_data = data[n:]





    for iter in range(ITERS):
        print(f"iter #{iter}")
        torch.cuda.empty_cache()
        # every once in a while evaluate the loss on train and val sets
        if iter % EVAL_INTERVALS == 0 or iter == ITERS - 1:
            losses = estimate_loss(m)
            liveloss.update({ 'loss': losses['train'], 'val_loss': losses['val']})
            liveloss.send()
            print("ITER:", i // step, "::::", " STRING INDEX:", i)
            print("tokens", len(data)/10**6,"M")
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, time {int((time.time() - startTime)//60)} minutes")
            
        torch.cuda.empty_cache()
        # sample a batch of data
        xb, yb = get_batch('train')

        # evaluate the loss
        logits, loss = m(xb, yb)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

    endTime = time.time()
    print(f"Total Training Time : {int((endTime - startTime)//60)} minutes")

In [ ]:
655,360,000
989,593,600

In [ ]:
input_tokens = torch.tensor(tokenizer.encode(" ")).unsqueeze(0).cuda()
# print(input_tokens)


output_tokens = m.generate(input_tokens, max_new_tokens=100)[0]
# print(output_tokens)

output : str = tokenizer.decode(output_tokens)



print(output)


In [ ]:

def convert_optimizer_state_to_float32(optimizer : torch.optim.AdamW):
    for state in optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor) and v.dtype == torch.float64:
                state[k] = v.float()


# Saving the model
def save_model(model : nn.Module, optimizer : torch.optim.AdamW, filepath):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(checkpoint, filepath)
    print(f"Model saved to {filepath}")

# Loading the model
def load_model(model : nn.Module, optimizer : torch.optim.AdamW, filepath, device):
    checkpoint = torch.load(filepath, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    convert_optimizer_state_to_float32(optimizer)

    print(f"Model loaded from {filepath}")
    model.to(device)
    return model, optimizer





In [ ]:
# Save after training
save_model(m, optimizer, 'weights/gpt_model_50M.pth') # TODO improve this naming convention

# Later or for inference
# model = GPTLanguageModel()
# optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
# model, optimizer = load_model(model, optimizer, 'gpt_model_checkpoint.pth', device)


In [ ]:
pre_model = GPTLanguageModel()
pre_optimizer = torch.optim.AdamW(pre_model.parameters(), lr=LEARNING_RATE)

loaded_model, loaded_optimizer = load_model(pre_model, pre_optimizer, './weights/gpt_model_50M.pth', device)


In [ ]:
loaded_model = loaded_model.cuda()

In [ ]:
ITERS = 100
startTime = time.time()
step = BATCH_SIZE * modelSpecs.BLOCK_SIZE * ITERS * 8
liveloss = PlotLosses()

# for i in range(step * 0, len(stories), step):
for i in range(989593600, len(stories), step):
    print("ITER:", i // step, "::::", " STRING INDEX:", i)
    text = stories[i : i + step]
    data = torch.tensor(tokenizer.encode(text))
    n = int(TRAIN_TEST_SPLIT * len(data))
    print("tokens", len(data)/10**6,"M")

    train_data = data[:n]
    val_data = data[n:]





    for iter in range(ITERS):
        print(f"iter #{iter}")
        torch.cuda.empty_cache()
        # every once in a while evaluate the loss on train and val sets
        if iter % EVAL_INTERVALS == 0 or iter == ITERS - 1:
            losses = estimate_loss(loaded_model)
            liveloss.update({ 'loss': losses['train'], 'val_loss': losses['val']})
            liveloss.send()
            print("ITER:", i // step, "::::", " STRING INDEX:", i)
            print("tokens", len(data)/10**6,"M")
            print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}, time {int((time.time() - startTime)//60)} minutes")
        torch.cuda.empty_cache()
        # sample a batch of data
        xb, yb = get_batch('train')

        # evaluate the loss
        
        logits, loss = loaded_model(xb, yb)
        loaded_optimizer.zero_grad(set_to_none=True)
        loss.backward()
        loaded_optimizer.step()

    endTime = time.time()
    print(f"Total Training Time : {int((endTime - startTime)//60)} minutes")

In [ ]:
input_tokens = torch.tensor(tokenizer.encode(" ")).unsqueeze(0).cuda()
# print(input_tokens)


output_tokens = loaded_model.generate(input_tokens, max_new_tokens=100)[0]
# print(output_tokens)

output : str = tokenizer.decode(output_tokens)



print(output)
